In [1]:
# Libraries
library(tidyverse) # veri işleme ve görselleştirme için
library(ggpubr) # kolayca yayına hazır grafikler oluşturma
library(rstatix) # kolay istatistiksel analizler için pipeline R işlevleri sağlar

# Veri Oluşturma

In [2]:
# Veri Oluşturma
set.seed(111) # Aynı sonuçları elde etmek istiyorsanız, aynı set seed değerini yazmalısınız.
dil_puanlari <- sample(x = 60:100, size = 30, replace = TRUE) # 60 ile 100 puan arası dağılım olsun

gruplar <- c(rep("birinci_grup", 10),
         rep("ikinci_grup", 10),
         rep("ucuncu_grup", 10))

basit_data = data.frame(gruplar, dil_puanlari) # serileri dataframe yapalım
basit_data

# Betimsel İstatistikler

In [3]:
basit_data %>%
  group_by(gruplar) %>%
  get_summary_stats(dil_puanlari, type = "mean_sd")

# Görselleştirme

In [4]:
# qplot görselleştirme
ggboxplot(basit_data, x = "gruplar", y = "dil_puanlari")

# Varsayımlar
## Aykırı Değerler

In [5]:
# Aykırı değerler, rstatix paketi içindeki define_outliers() fonksiyonuyla kutu çizim yöntemleri kullanılarak kolayca tanımlanabilir.
basit_data %>% 
  group_by(gruplar) %>%
  identify_outliers(dil_puanlari)

Aykırı değer bulunmamaktadır.

## Normallik Varsayımı
Normallik varsayımı, aşağıdaki iki yaklaşımdan biri kullanılarak kontrol edilebilir:

1. Tüm gruplar için normalliği kontrol etmek için ANOVA model artıklarının birlikte analiz edilmesini içerir. Bu yaklaşım daha kolaydır ve çok sayıda grubunuz olduğunda veya grup başına birkaç veri noktası (gözlem) olduğunda çok kullanışlıdır.

2. Her grup için ayrı ayrı normalliği kontrol edin. Bu yaklaşım, yalnızca birkaç grubunuz ve grup başına çok sayıda veri noktanız (gözlem) olduğunda kullanılabilir.

Hem seçenek 1 hem de seçenek 2'yi yapalım.

Model artıklarını analiz ederek normallik varsayımını kontrol edin. QQ grafiği ve Shapiro-Wilk normallik testi kullanılmıştır. QQ grafiği, belirli bir veri ile normal dağılım arasındaki korelasyonu çizer.

In [6]:
# Build the linear model
model  <- lm(dil_puanlari ~ gruplar, data = basit_data)
# Create a QQ plot of residuals
ggqqplot(residuals(model))

In [7]:
# Shapiro-Wilk normallik testi
shapiro_test(residuals(model))

QQ grafiğinde, tüm noktalar yaklaşık olarak referans çizgisi boyunca düştüğü için normallik varsayabiliriz. Bu sonuç Shapiro-Wilk testi tarafından desteklenmektedir. p değeri anlamlı değildir (p = 0.45), dolayısıyla normalliği varsayabiliriz.

Gruplara göre normallik varsayımını kontrol ediyoruz. Her grup seviyesi için Shapiro-Wilk testinin hesaplanması. Veriler normal dağılıyorsa, p değeri 0,05'ten büyük olmalıdır.

In [8]:
basit_data %>%
  group_by(gruplar) %>%
  shapiro_test(dil_puanlari)

Shapiro-Wilk'in normallik testi ile değerlendirildiği üzere, her grup için normal olarak dağılmıştır (p > 0.05).


Örnek boyutunuz 50'den büyükse, normal QQ grafiğinin tercih edildiğini unutmayın çünkü daha büyük örnek boyutlarında Shapiro-Wilk testi normallikten küçük bir sapmaya bile çok duyarlı hale gelir.

QQ grafiği, belirli bir veri ile normal dağılım arasındaki korelasyonu çizer. Her grup seviyesi için QQ grafikleri oluşturun:

In [9]:
ggqqplot(basit_data, "dil_puanlari", facet.by = "gruplar")

Tüm noktalar, her hücre için yaklaşık olarak referans çizgisi boyunca yer almaktadır. Böylece verilerin normal dağıldığını varsayabiliriz.

**Not:** Verilerin normalliği konusunda şüpheniz varsa tek yönlü ANOVA testinin parametrik olmayan alternatifi olan Kruskal-Wallis testini kullanabilirsiniz.

## Varyans Homejenliği
1. Artıklar-uyum grafiği, varyansların homojenliğini kontrol etmek için kullanılabilir.

In [10]:
plot(model, 1)

Yukarıdaki grafikte, artıklar ve uygun değerler (her grubun ortalaması) arasında belirgin bir ilişki yoktur ki bu iyidir. Dolayısıyla varyansların homojenliğini varsayabiliriz.

Varyansların homojenliğini kontrol etmek için Levene testini kullanmak da mümkündür:

In [11]:
# Levene Testi
basit_data %>% levene_test(dil_puanlari ~ gruplar)

Yukarıdaki çıktıdan, p-değerinin > 0.05 olduğunu görebiliriz, bu da anlamlı değildir. Bu, gruplar arasındaki varyanslar arasında anlamlı bir fark olmadığı anlamına gelir. Bu nedenle,  grup varyans homojenliği de sağlanmaktadır.

Varyans homojenliği varsayımının karşılanmadığı bir durumda, welch_anova_test()[rstatix paketi] fonksiyonunu kullanarak Welch tek yönlü ANOVA testini kullanabilirsiniz. Bu test eşit varyans varsayımını gerektirmez.

In [12]:
res.aov <- basit_data %>% anova_test(dil_puanlari ~ gruplar)
res.aov


p değeri p > 0.05'den büyük olduğu için gruplar arası anlamlı bir farklılık olmadığı görülmektedir.

Yukarıdaki tabloda, ges sütunu genelleştirilmiş eta kareyi yani etki büyüklüğüne karşılık gelmektedir. Sonuç değişkenindeki (dil puani) tahmin edici (grubu) açısından açıklanabilen değişkenliğin oranını ölçer. 0,18 (%18)'lik bir etki büyüklüğü, ağırlıktaki değişimin %18'nin dil puanı için açıklanabileceği anlamına gelir.

# Kaynakça
* https://statsandr.com/blog/anova-in-r/
* https://www.datanovia.com/en/lessons/anova-in-r/
* İstatistikten Nefret Edenler İçin İstatistik - Neil Salkind